# **Processing EARS and TIMIT dataset for low-level features**
**ABOUT DATASET:**
1. Make sure TIMIT dataset is predownloaded to google drive, copy its path for future usage.
2. EARS dataset will be downloaded in temporary storage space when you go through this notebook.
3. All EARS speakers (p001–p107) are used exclusively for training, ensuring full consistency with the experimental setup of the TEARS dataset.

**ABOUT FEATURES:**

3. Low-level features contains `pitch_mean, pitch_median, pitch_range, pitch_std, pitch_variance, F1/F2/F3_median, F1/F2/F3_mean, F1/F2/F3_std, hnr_mean, energy_max, energy_min, energy_mean, energy_std, energy_dynamic_range`, 20 in total.

**ABOUT OUTPUT FORMAT:**

4. Output csv file with audio file information and feature metadata.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%bash
set -euo pipefail

ROOT=/content/EARS_dataset
mkdir -p "$ROOT"
cd "$ROOT"

ZIP_BASE_URL="https://github.com/facebookresearch/ears_dataset/releases/download/dataset"
META_BASE_URL="https://raw.githubusercontent.com/facebookresearch/ears_dataset/main"

echo "=== Checking and downloading EARS dataset (ALL speakers 001–107 as TRAIN) ==="

download_and_extract() {
  local X=$1
  local ZIP="p${X}.zip"
  local DIR="p${X}"

  if [ -d "$DIR" ]; then
    n=$(find "$DIR" -type f -name '*.wav' | wc -l)
    if [ "$n" -gt 10 ]; then
      echo "-- p${X}: already has $n wav files, skip."
      return
    else
      echo "-- p${X}: incomplete ($n files), re-downloading."
      rm -rf "$DIR"
    fi
  fi

  echo "-- Downloading p${X}.zip ..."
  curl -L -C - --retry 5 --retry-delay 5 -o "$ZIP" "$ZIP_BASE_URL/$ZIP" || {
    echo "!! Failed to download p${X}.zip"; return;
  }

  if unzip -tq "$ZIP" >/dev/null 2>&1; then
    echo "   OK: $ZIP valid. Extracting..."
  else
    echo "   Corrupt zip $ZIP, will retry."
    rm -f "$ZIP"
    return
  fi

  unzip -oq "$ZIP"
}

echo "=== Download ALL speakers (001–107) ==="
for X in $(seq -w 001 107); do
  download_and_extract "$X"
done

# === Extra metadata files ===
echo "=== Download metadata JSONs ==="
curl -L --retry 5 --retry-delay 5 \
  -o speaker_statistics.json \
  "$META_BASE_URL/speaker_statistics.json"

curl -L --retry 5 --retry-delay 5 \
  -o transcripts.json \
  "$META_BASE_URL/transcripts.json"

echo "=== JSON file heads ==="
head -n 5 speaker_statistics.json
head -n 5 transcripts.json


=== Checking and downloading EARS dataset (ALL speakers 001–107 as TRAIN) ===
=== Download ALL speakers (001–107) ===
-- p001: already has 161 wav files, skip.
-- p002: already has 161 wav files, skip.
-- Downloading p003.zip ...
   OK: p003.zip valid. Extracting...
-- Downloading p004.zip ...
   OK: p004.zip valid. Extracting...
-- Downloading p005.zip ...
   OK: p005.zip valid. Extracting...
-- Downloading p006.zip ...
   OK: p006.zip valid. Extracting...
-- Downloading p007.zip ...
   OK: p007.zip valid. Extracting...
-- Downloading p008.zip ...
   OK: p008.zip valid. Extracting...
-- Downloading p009.zip ...
   OK: p009.zip valid. Extracting...
-- Downloading p010.zip ...
   OK: p010.zip valid. Extracting...
-- Downloading p011.zip ...
   OK: p011.zip valid. Extracting...
-- Downloading p012.zip ...
   OK: p012.zip valid. Extracting...
-- Downloading p013.zip ...
   OK: p013.zip valid. Extracting...
-- Downloading p014.zip ...
   OK: p014.zip valid. Extracting...
-- Downloading p01

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  621M  100  621M    0     0  98.0M      0  0:00:06  0:00:06 --:--:--  112M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  583M  100  583M    0     0   155M      0  0:00:03  0:00:03 --:--:--  197M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  602M  100  602M    0     0   153M      0  0:00:03  0:00:03 --:--:--  182M
  % Total    % Received % Xferd  Average Speed   Tim

In [ ]:
# TODO: download the VoxCeleb_dataset

In [ ]:
# VOX_NATIONALITY_TO_ACCENT = {
#     'USA': 'american',
#     'UK': 'british',
#     'Australia': 'australian',
#     'Canada': 'canadian',
#     'India': 'indian',
#     'Ireland': 'irish',
#     'South Africa': 'south_african',
#     'New Zealand': 'new_zealand',
#     'Scotland': 'scottish',
#     'Wales': 'welsh'
# }
# # Major English-speaking countries to keep
# MAJOR_ENGLISH_COUNTRIES = list(VOX_NATIONALITY_TO_ACCENT.keys())

In [ ]:
TIMIT_DIALECT_MAP = {
    'dr1': 'new_england',
    'dr2': 'northern',
    'dr3': 'north_midland',
    'dr4': 'south_midland',
    'dr5': 'southern',
    'dr6': 'new_york_city',
    'dr7': 'western',
    'dr8': 'army_brat'
}

In [ ]:
import json
from pathlib import Path

stats_path = Path("/content/EARS_dataset/speaker_statistics.json") # EARS speaker_statistics.json has speaker information(including native_language)
with open(stats_path, "r") as f:
    ears_meta = json.load(f)

ears_native_lang = {
    spk: info.get("native language", "")
    for spk, info in ears_meta.items()
}

list(ears_native_lang.items())[:5]

[('p001', 'german'),
 ('p002', 'american english'),
 ('p003', 'american english'),
 ('p004', 'american english'),
 ('p005', 'mandarin')]

In [ ]:
ears_native_lang = {
    spk: info.get("native language", "")
    for spk, info in ears_meta.items()
}

In [ ]:
!pip install praat-parselmouth

In [ ]:
!pip install librosa

In [ ]:
import numpy as np
import pandas as pd
import re
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean, cdist
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy.stats import f_oneway, kruskal

import parselmouth
import librosa
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import time

In [ ]:
def extract_comprehensive_features(audio_path, target_sr=16000):
    """
    Extract comprehensive acoustic features from audio file

    Args:
        audio_path: Path to audio file (wav/flac/mp3)
        target_sr: Target sampling rate (default: 16000 Hz)

    Returns:
        Dictionary of acoustic features (20 features + metadata)
    """
    try:
        # Uniform resampling to 16kHz mono
        y, sr = librosa.load(audio_path, sr=target_sr, mono=True)
        sound = parselmouth.Sound(y, sampling_frequency=target_sr)

        features = {}

        # Basic audio information
        features['duration'] = sound.duration
        features['sampling_frequency'] = target_sr

        # ===== 1. Pitch features (5 features) =====
        pitch = sound.to_pitch(
            time_step=0.01,      # Add time_step for consistency
            pitch_floor=75.0,
            pitch_ceiling=600.0
        )

        pitch_values = pitch.selected_array['frequency']
        pitch_values = pitch_values[pitch_values > 0]  # Filter unvoiced frames

        if len(pitch_values) > 0:
            features['pitch_mean'] = np.mean(pitch_values)
            features['pitch_std'] = np.std(pitch_values)
            features['pitch_median'] = np.median(pitch_values)
            features['pitch_range'] = np.ptp(pitch_values)  # max - min
            features['pitch_variance'] = np.var(pitch_values)
        else:
            features['pitch_mean'] = np.nan
            features['pitch_std'] = np.nan
            features['pitch_median'] = np.nan
            features['pitch_range'] = np.nan
            features['pitch_variance'] = np.nan

        # ===== 2. Energy features (5 features) =====
        intensity = sound.to_intensity(
            minimum_pitch=75.0,
            time_step=0.01      # Add time_step for consistency
        )

        intensity_values = intensity.values[0]
        intensity_values = intensity_values[~np.isnan(intensity_values)]

        if len(intensity_values) > 0:
            features['energy_mean'] = np.mean(intensity_values)
            features['energy_std'] = np.std(intensity_values)
            features['energy_max'] = np.max(intensity_values)
            features['energy_min'] = np.min(intensity_values)
            features['energy_dynamic_range'] = features['energy_max'] - features['energy_min']
        else:
            features['energy_mean'] = np.nan
            features['energy_std'] = np.nan
            features['energy_max'] = np.nan
            features['energy_min'] = np.nan
            features['energy_dynamic_range'] = np.nan

        # ===== 3. Formant features (9 features: F1/F2/F3 x 3 stats) =====
        formant = sound.to_formant_burg(
            time_step=0.01,              # Add time_step
            max_number_of_formants=5,
            maximum_formant=5500,        # Uniform parameter
            window_length=0.025,         # 25ms window
            pre_emphasis_from=50.0
        )

        formant_times = formant.ts()

        # Extract F1, F2, F3
        for i in range(1, 4):  # F1, F2, F3
            formant_values = []

            for t in formant_times:
                f = formant.get_value_at_time(i, t)
                if not np.isnan(f) and f > 0:
                    formant_values.append(f)

            if len(formant_values) > 0:
                features[f'F{i}_mean'] = np.mean(formant_values)
                features[f'F{i}_std'] = np.std(formant_values)
                features[f'F{i}_median'] = np.median(formant_values)
            else:
                features[f'F{i}_mean'] = np.nan
                features[f'F{i}_std'] = np.nan
                features[f'F{i}_median'] = np.nan

        # ===== 4. HNR feature (1 feature) =====
        try:
            harmonicity = parselmouth.praat.call(
                sound,
                "To Harmonicity (cc)",
                0.01,   # time_step
                75.0,   # minimum pitch
                0.1,    # silence threshold
                1.0     # periods per window
            )

            hnr_values = harmonicity.values[0]
            hnr_values = hnr_values[~np.isnan(hnr_values)]
            hnr_values = hnr_values[hnr_values != -200]  # Remove undefined values

            if len(hnr_values) > 0:
                features['hnr_mean'] = np.mean(hnr_values)
            else:
                features['hnr_mean'] = np.nan
        except Exception as e:
            print(f"Warning: HNR extraction failed for {audio_path}: {e}")
            features['hnr_mean'] = np.nan

        return features

    except Exception as e:
        print(f"Error processing file {audio_path}: {e}")
        return None

In [ ]:
# *** Parse EARS emotion/type from filename: emo_<emotion>_<type>.wav
def parse_ears_filename(fname: str):
    m = re.match(r"emo_(\w+)_(\w+)\.", fname.lower())
    if m:
        return m.group(1), m.group(2)  # (emotion, speech_type)
    return "", ""

In [ ]:
# *** Parse TIMIT fields from path: .../train/dr4/fmld0/sa1.wav
def parse_timit_path(path: Path):
    # original_name = path.name

    pstr = str(path).lower().replace("\\", "/")

    m = re.search(r"/(dr[1-8])/([^/]+)/[^/]+$", pstr)
    if m:
        dialect_code = m.group(1)              # dr4
        speaker_id   = m.group(2)              # fmld0
    else:
        m_dr = re.search(r"/(dr[1-8])/", pstr)
        dialect_code = m_dr.group(1) if m_dr else ""
        speaker_id = path.parent.name.lower()

    dialect = TIMIT_DIALECT_MAP.get(dialect_code.lower(), "")

    if "/train/" in pstr:
        split = "train"
    elif "/test/" in pstr:
        split = "test"
    else:
        split = ""

    return speaker_id, dialect_code, dialect, split

In [ ]:
# # *** Load VoxCeleb2 metadata and build speaker -> accent map
# def build_vox_speaker_accent_map(vox_meta_csv: str):
#     spk2acc = {}
#     if not vox_meta_csv or not os.path.exists(vox_meta_csv):
#         return spk2acc
#     df = pd.read_csv(vox_meta_csv)
#     # Expect typical columns: 'VoxCeleb2 ID','VGGFace2 ID','Gender','Nationality','Set'
#     id_col = None
#     for c in df.columns:
#         if str(c).lower().strip().startswith("voxceleb") and "id" in str(c).lower():
#             id_col = c; break
#     nat_col = None
#     for c in df.columns:
#         if "nationality" in str(c).lower():
#             nat_col = c; break
#     if id_col is None or nat_col is None:
#         return spk2acc
#     for _, r in df.iterrows():
#         spk = str(r[id_col]).strip()
#         nat = str(r[nat_col]).strip()
#         acc = VOX_NATIONALITY_TO_ACCENT.get(nat, "").lower()
#         spk2acc[spk] = acc
#     return spk2acc


In [ ]:
COMMON_META_COLS = [
    "filename",
    "speaker_id",
    "dataset",
    "file_path",
    "dialect_code",
    "dialect",
    "split",
    "emotion",
    "speech_type",
    "accent",
    "native_language",
]

In [ ]:
def ensure_common_schema(df: pd.DataFrame) -> pd.DataFrame:
    for col in COMMON_META_COLS:
        if col not in df.columns:
            df[col] = ""
    meta_cols = [c for c in COMMON_META_COLS if c in df.columns]
    other_cols = [c for c in df.columns if c not in meta_cols]
    return df[meta_cols + other_cols]

In [ ]:
def _process_with_label_parser(
    data_path,
    dataset_name,
    target_sr,
    file_extensions,
    max_files_per_speaker,
    speaker_id_fn,
    label_parser_fn,
):
    data_path = Path(data_path)
    assert data_path.exists(), f"{data_path} not found"

    audio_files = []
    for ext in file_extensions:
        audio_files += list(data_path.rglob(f"*.{ext}"))
    audio_files = sorted(audio_files)

    print(f"\nProcessing {dataset_name}")
    print(f"Found {len(audio_files)} audio files\n")

    speakers = {}
    for f in audio_files:
        spk = speaker_id_fn(f)
        speakers.setdefault(spk, []).append(f)

    print(f"Detected {len(speakers)} speakers")

    all_feats, info_rows = [], []

    for speaker_id, files in speakers.items():
        files = sorted(files)
        if max_files_per_speaker is not None:
            files = files[:max_files_per_speaker]

        print(f"Speaker {speaker_id}: {len(files)} files")

        for audio_file in tqdm(files, desc=f"  Processing {speaker_id}"):
            meta = label_parser_fn(audio_file, speaker_id)

            feats = extract_comprehensive_features(str(audio_file), target_sr=target_sr)
            if feats is None:
                continue

            feats["filename"] = audio_file.name
            feats["speaker_id"] = speaker_id
            feats["dataset"] = dataset_name
            feats["file_path"] = str(audio_file)

            for k, v in meta.items():
                feats[k] = v

            all_feats.append(feats)

            info_row = {
                "filename": audio_file.name,
                "speaker_id": speaker_id,
                "dataset": dataset_name,
                "file_path": str(audio_file),
            }
            info_row.update(meta)
            info_rows.append(info_row)

    features_df = pd.DataFrame(all_feats)
    info_df = pd.DataFrame(info_rows)

    features_df = ensure_common_schema(features_df)
    info_df = ensure_common_schema(info_df)

    return features_df, info_df

In [ ]:
def timit_speaker_id(path: Path) -> str:
    spk, _, _, _ = parse_timit_path(path)
    return spk

def timit_label_parser(path: Path, speaker_id: str) -> dict:
    _, dialect_code, dialect, split = parse_timit_path(path)
    return {
        "dialect_code": dialect_code,
        "dialect": dialect,
        "split": split,
        "emotion": "",
        "speech_type": "",
        "accent": "",
        "native_language": "american_english",
    }

def process_timit_dataset(
    data_path,
    target_sr=16000,
    file_extensions=("wav",),
    max_files_per_speaker=None,
    dataset_name="TIMIT",
):
    return _process_with_label_parser(
        data_path=data_path,
        dataset_name=dataset_name,
        target_sr=target_sr,
        file_extensions=file_extensions,
        max_files_per_speaker=max_files_per_speaker,
        speaker_id_fn=timit_speaker_id,
        label_parser_fn=timit_label_parser,
    )


In [ ]:
def ears_speaker_id(path: Path) -> str:
    # p001/emo_xxx.wav
    return path.parent.name

def parse_ears_filename(fname: str):
    m = re.match(r"emo_(\w+)_(\w+)\.", fname.lower())
    if m:
        return m.group(1), m.group(2)
    return "", ""

def make_ears_label_parser(ears_native_lang: dict, split: str = "train"):
    def _parser(path: Path, speaker_id: str) -> dict:
        emotion, speech_type = parse_ears_filename(path.name)
        native_language = ""
        if ears_native_lang is not None:
            native_language = ears_native_lang.get(speaker_id, "")
        return {
            "dialect_code": "",
            "dialect": "",
            "split": split,
            "emotion": emotion,
            "speech_type": speech_type,
            "accent": "",
            "native_language": native_language,
        }
    return _parser

def process_ears_dataset(
    data_path,
    ears_native_lang: dict = None,
    target_sr=16000,
    file_extensions=("wav",),
    max_files_per_speaker=None,
    dataset_name="EARS",
    split="train",
):
    label_parser = make_ears_label_parser(ears_native_lang, split=split)
    return _process_with_label_parser(
        data_path=data_path,
        dataset_name=dataset_name,
        target_sr=target_sr,
        file_extensions=file_extensions,
        max_files_per_speaker=max_files_per_speaker,
        speaker_id_fn=ears_speaker_id,
        label_parser_fn=label_parser,
    )


In [ ]:
# Process EARS dataset
ears_df, ears_info = process_ears_dataset(
    "/content/EARS_dataset",
    ears_native_lang=ears_native_lang,
    split="train",
)


Processing EARS
Found 17227 audio files

Detected 107 speakers
Speaker p001: 161 files


  Processing p001: 100%|██████████| 161/161 [00:55<00:00,  2.88it/s]


Speaker p002: 161 files


  Processing p002: 100%|██████████| 161/161 [00:52<00:00,  3.08it/s]


Speaker p003: 161 files


...

  Processing p105: 100%|██████████| 161/161 [00:59<00:00,  2.69it/s]


Speaker p106: 161 files


  Processing p106: 100%|██████████| 161/161 [01:06<00:00,  2.41it/s]


Speaker p107: 161 files


  Processing p107: 100%|██████████| 161/161 [00:58<00:00,  2.75it/s]


In [ ]:
# Save EARS results
ears_df.to_csv('/content/drive/MyDrive/ears_features.csv', index=False)
ears_info.to_csv('/content/drive/MyDrive/ears_info.csv', index=False)

print("\nFiles saved: ears_features.csv, ears_info.csv")


Files saved: ears_features.csv, ears_info.csv


In [ ]:
# process TIMIT dataset
timit_df, timit_info = process_timit_dataset("/content/drive/MyDrive/TIMIT/TRAIN", file_extensions=("WAV", "wav")) # use pre-downloaded dataset path


Processing TIMIT
Found 4620 audio files

Detected 462 speakers
Speaker fcjf0: 10 files


  Processing fcjf0: 100%|██████████| 10/10 [00:00<00:00, 16.57it/s]


Speaker fdaw0: 10 files


  Processing fdaw0: 100%|██████████| 10/10 [00:00<00:00, 13.53it/s]


Speaker fdml0: 10 files


...

  Processing mrlk0: 100%|██████████| 10/10 [00:03<00:00,  3.04it/s]


Speaker mrre0: 10 files


  Processing mrre0: 100%|██████████| 10/10 [00:03<00:00,  2.93it/s]


Speaker mtcs0: 10 files


  Processing mtcs0: 100%|██████████| 10/10 [00:03<00:00,  3.18it/s]


In [ ]:
# Save TIMIT results
timit_df.to_csv('/content/drive/MyDrive/TIMIT_features.csv', index=False)
timit_info.to_csv('/content/drive/MyDrive/TIMIT_info.csv', index=False)

print("\nFiles saved: TIMIT_features.csv, TIMIT_info.csv")


Files saved: TIMIT_features.csv, TIMIT_info.csv


In [3]:
# check dataset sample number
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/ears_features.csv")
len(df)

17227

In [4]:
df = pd.read_csv("/content/drive/MyDrive/TIMIT_features.csv")
len(df)

4620